# 사용자 성향 코사인 유사도분석기반의 협업필터링 여행지 추천

## 드라이브 연결 및 모듈 불러오기, 초기설정

In [ ]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


# 한글 호환 문제
from matplotlib import font_manager as fm
from matplotlib import rc

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rcParams['font.family'] = 'NanumBarunGothic'
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rcParams['font.family'] = font_name

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,684 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121749 files and dire

In [ ]:
pd.set_option('display.max_columns', None)

## 데이터 불러오기

In [ ]:
recom_data = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/004.전처리된데이터/여행객.csv')

In [ ]:
recom_data.head()

,TRAVELER_ID,성별,연령,거주지,MBTI,여행빈도,여행빈도(기간),여행현황(동반),여행동반자수,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,여행동기1,여행동기2,여행동기3,선호여행1,선호여행2,선호여행3
0,a006685,여,30,경기도,INFP,6,1년,2인 가족 여행,1,전자 매우선호,전자 매우선호,후자 매우선호,후자 조금선호,전자 매우선호,전자 매우선호,후자 조금선호,전자 중간선호,기타,NaN,NaN,포천시,인천광역시,영월군
1,a014274,여,20,서울특별시,ESTJ,5,1년,나홀로 여행,0,전자 중간선호,전자 매우선호,전자 매우선호,후자 조금선호,전자 조금선호,전자 조금선호,전자 중간선호,후자 중간선호,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",제주특별자치도,경주시,원주시
2,a001158,남,30,경기도,ENFJ,2,한달,나홀로 여행,0,전자 매우선호,전자 조금선호,전자 중간선호,전자 조금선호,전자 중간선호,전자 중간선호,전자 조금선호,전자 중간선호,"역사 탐방, 문화적 경험 등 교육적 동기","일상적인 환경 및 역할에서의 탈출, 지루함 탈피","운동, 건강 증진 및 충전",서울특별시,창원시,제주특별자치도
3,a005594,남,30,인천광역시,ENFP,2,한달,자녀 동반 여행,3,전자 중간선호,후자 매우선호,전자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,전자 중간선호,여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",서울특별시,인천광역시,김포시
4,a007033,남,20,서울특별시,ENFP,1,1주일,나홀로 여행,0,후자 매우선호,후자 매우선호,전자 매우선호,전자 매우선호,전자 조금선호,전자 매우선호,후자 매우선호,후자 매우선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,하남시,서울특별시,서울특별시


## 데이터 전처리

In [ ]:
# 여행스타일 라벨링
def merge_travelstyle(df, style_col, col_A, col_B):
    df[col_A] = 0
    df[col_B] = 0

    df.loc[df[style_col].isin(['전자 매우선호', '전자 중간선호', '전자 조금선호']), col_A] = 1
    df.loc[df[style_col].isin(['후자 조금선호', '후자 중간선호', '후자 매우선호']), col_B] = 1
    df.drop(style_col, axis=1, inplace=True)
    return df

In [ ]:
recom_data = merge_travelstyle(recom_data, '자연/도시', '자연', '도시')
recom_data = merge_travelstyle(recom_data, '숙박/당일', '숙박', '당일')
recom_data = merge_travelstyle(recom_data, '익숙한/새로운', '익숙한', '새로운')
recom_data = merge_travelstyle(recom_data, '편하지만비싼숙소/불편하지만저렴한숙소', '편하지만비싼숙소', '불편하지만저렴한숙소')
recom_data = merge_travelstyle(recom_data, '휴양휴식/체험활동', '휴양휴식', '체험활동')
recom_data = merge_travelstyle(recom_data, '잘알려지지않은/알려진', '잘알려지지않은', '알려진')
recom_data = merge_travelstyle(recom_data, '계획에따른/상황에따른', '계획에따른', '상황에따른')
recom_data = merge_travelstyle(recom_data, '사진촬영중요하지않음/사진촬영중요', '사진촬영중요하지않음', '사진촬영중요')

# 성별, MBTI 원핫인코딩
recom_data = pd.get_dummies(recom_data, columns=['성별', 'MBTI',])

# 연령 라벨링
recom_data['연령'] = recom_data['연령'].astype(str).str[0]

# 컬럼 순서 조정
recom_data = recom_data[['TRAVELER_ID', '연령', '성별_남', '성별_여', '자연', '도시', '숙박', '당일', '익숙한', '새로운',
                    '편하지만비싼숙소', '불편하지만저렴한숙소', '휴양휴식', '체험활동', '잘알려지지않은', '알려진', '계획에따른',
                    '상황에따른', '사진촬영중요하지않음', '사진촬영중요', 'MBTI_ENFJ',
                    'MBTI_ENFP', 'MBTI_ENTJ', 'MBTI_ENTP', 'MBTI_ESFJ', 'MBTI_ESFP',
                    'MBTI_ESTJ', 'MBTI_ESTP', 'MBTI_INFJ', 'MBTI_INFP', 'MBTI_INTJ',
                    'MBTI_INTP', 'MBTI_ISFJ', 'MBTI_ISFP', 'MBTI_ISTJ', 'MBTI_ISTP', '선호여행1', '선호여행2', '선호여행3']]

recom_data.head()

,TRAVELER_ID,연령,성별_남,성별_여,자연,도시,숙박,당일,익숙한,새로운,편하지만비싼숙소,불편하지만저렴한숙소,휴양휴식,체험활동,잘알려지지않은,알려진,계획에따른,상황에따른,사진촬영중요하지않음,사진촬영중요,MBTI_ENFJ,MBTI_ENFP,MBTI_ENTJ,MBTI_ENTP,MBTI_ESFJ,MBTI_ESFP,MBTI_ESTJ,MBTI_ESTP,MBTI_INFJ,MBTI_INFP,MBTI_INTJ,MBTI_INTP,MBTI_ISFJ,MBTI_ISFP,MBTI_ISTJ,MBTI_ISTP,선호여행1,선호여행2,선호여행3
0,a006685,3,0,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,포천시,인천광역시,영월군
1,a014274,2,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,제주특별자치도,경주시,원주시
2,a001158,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,서울특별시,창원시,제주특별자치도
3,a005594,3,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,서울특별시,인천광역시,김포시
4,a007033,2,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,하남시,서울특별시,서울특별시


### 데이터 저장

In [ ]:
recom_data.to_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/004.전처리된데이터/추천시스템전처리.csv')

### 관광객 성향(유사도 계산용)

In [ ]:
traveler_rev = recom_data.iloc[:, :-3]
traveler_rev.head()

,TRAVELER_ID,연령,성별_남,성별_여,자연,도시,숙박,당일,익숙한,새로운,편하지만비싼숙소,불편하지만저렴한숙소,휴양휴식,체험활동,잘알려지지않은,알려진,계획에따른,상황에따른,사진촬영중요하지않음,사진촬영중요,MBTI_ENFJ,MBTI_ENFP,MBTI_ENTJ,MBTI_ENTP,MBTI_ESFJ,MBTI_ESFP,MBTI_ESTJ,MBTI_ESTP,MBTI_INFJ,MBTI_INFP,MBTI_INTJ,MBTI_INTP,MBTI_ISFJ,MBTI_ISFP,MBTI_ISTJ,MBTI_ISTP
0,a006685,3,0,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,a014274,2,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,a001158,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,a005594,3,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,a007033,2,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 관광객별 선호여행지 3개

In [ ]:
like_city = recom_data.iloc[:, [0] + list(range(-3, 0))]
like_city.head()

,TRAVELER_ID,선호여행1,선호여행2,선호여행3
0,a006685,포천시,인천광역시,영월군
1,a014274,제주특별자치도,경주시,원주시
2,a001158,서울특별시,창원시,제주특별자치도
3,a005594,서울특별시,인천광역시,김포시
4,a007033,하남시,서울특별시,서울특별시


## 코사인 유사도 계산 및 유사 사용자 선호 여행지 추천

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 코사인 유사도 계산
X = traveler_rev.iloc[:,1:].values
similarity_matrix = cosine_similarity(X)

# 유사도 행렬의 결과를 데이터프레임으로 변환
similarity_df = pd.DataFrame(similarity_matrix, index=traveler_rev['TRAVELER_ID'], columns=traveler_rev['TRAVELER_ID'])

# 가장 유사한 5명의 사용자 선호 여행지 추출
def get_user_preference(target_id, top_similar_ids, preference_df):
    # 타겟 사용자의 선호 여행지 목록
    target_cities = preference_df.loc[preference_df['TRAVELER_ID'] == target_id,
                                      ['선호여행1', '선호여행2', '선호여행3']].values.flatten()

    # 상위 유사 사용자들의 선호 여행지 목록
    similar_users_cities = preference_df[preference_df['TRAVELER_ID'].isin(top_similar_ids)]

    # 상위 유사 사용자들의 선호 여행지 중 타겟 사용자의 선호 여행지와 겹치지 않는 것만 추출
    recommended_cities_list = []
    for _, row in similar_users_cities.iterrows():
        cities = row[['선호여행1', '선호여행2', '선호여행3']].tolist()
        recommended_cities_list.extend([city for city in cities if city not in target_cities])

    # 중복된 여행지를 제거하고 추천 여행지 리스트 반환
    return list(set(recommended_cities_list))

# 특정 사용자에게 여행지 추천
def recommend_for_user(target_id, traveler_df, like_city_df):
    # 대상 사용자와 다른 모든 사용자 간의 유사도 가져오기
    target_similarity = similarity_df.loc[target_id]

    # 자기 자신을 제외하고, 가장 유사도가 높은 상위 5명의 사용자 찾기
    top_5_similar_ids = target_similarity.drop(target_id).nlargest(5).index

    # 추천 여행지 도출
    # 5명의 유사 사용자 배열
    top_5_ids = top_5_similar_ids.values
    # get_user_preference 함수사용,
    recommended_cities = get_user_preference(target_id, top_5_ids, like_city_df)

    # 결과 반환
    return recommended_cities

In [ ]:
similarity_df

TRAVELER_ID   a006685   a014274   a001158   a005594   a007033   a008021  \
TRAVELER_ID                                                               
a006685      1.000000  0.735767  0.736842  0.684211  0.551825  0.809858   
a014274      0.735767  1.000000  0.735767  0.551825  0.571429  0.786214   
a001158      0.736842  0.735767  1.000000  0.684211  0.674453  0.809858   
a005594      0.684211  0.551825  0.684211  1.000000  0.674453  0.584898   
a007033      0.551825  0.571429  0.674453  0.674453  1.000000  0.628971   
...               ...       ...       ...       ...       ...       ...   
d000922      0.736788  0.677631  0.891902  0.698010  0.722806  0.861892   
d010626      0.764866  0.628971  0.674882  0.809858  0.576557  0.692308   
d001143      0.736842  0.735767  0.736842  0.684211  0.674453  0.809858   
b004853      0.736842  0.858395  0.842105  0.578947  0.674453  0.899843   
d000216      0.789474  0.797081  0.789474  0.631579  0.735767  0.854850   

TRAVELER_ID   a007957   a007329   c002035   a004399   a002197   a006822  \
TRAVELER_ID                                                               
a006685      0.789474  0.735767  0.613139  0.719874  0.684211  0.735767   
a014274      0.797081  0.571429  0.785714  0.681385  0.674453  0.714286   
a001158      0.789474  0.674453  0.735767  0.719874  0.789474  0.674453   
a005594      0.736842  0.735767  0.674453  0.899843  0.631579  0.613139   
a007033      0.735767  0.785714  0.571429  0.681385  0.613139  0.642857   
...               ...       ...       ...       ...       ...       ...   
d000922      0.775567  0.722806  0.722806  0.762443  0.891902  0.632456   
d010626      0.719874  0.681385  0.628971  0.846154  0.809858  0.681385   
d001143      0.789474  0.551825  0.613139  0.809858  0.789474  0.797081   
b004853      0.789474  0.551825  0.735767  0.719874  0.789474  0.797081   
d000216      0.842105  0.613139  0.674453  0.764866  0.736842  0.858395   

TRAVELER_ID   a003223   a017742   a006017   a006061   a004208   a003110  \
TRAVELER_ID                                                               
a006685      0.613139  0.631579  0.814345  0.613139  0.490511  0.842105   
a014274      0.642857  0.735767  0.722806  0.785714  0.642857  0.674453   
a001158      0.858395  0.736842  0.853123  0.735767  0.735767  0.684211   
a005594      0.674453  0.684211  0.736788  0.551825  0.674453  0.736842   
a007033      0.571429  0.551825  0.767982  0.571429  0.571429  0.613139   
...               ...       ...       ...       ...       ...       ...   
d000922      0.767982  0.775567  0.885714  0.677631  0.677631  0.775567   
d010626      0.628971  0.764866  0.762443  0.628971  0.628971  0.899843   
d001143      0.735767  0.842105  0.853123  0.735767  0.735767  0.684211   
b004853      0.735767  0.842105  0.853123  0.919709  0.797081  0.684211   
d000216      0.674453  0.789474  0.891902  0.797081  0.674453  0.736842   

TRAVELER_ID   a001971   b005303   a000014   a003176   a002237   a005736  \
TRAVELER_ID                                                               
a006685      0.674453  0.674453  0.736788  0.789474  0.919709  0.809858   
a014274      0.714286  0.642857  0.677631  0.674453  0.642857  0.681385   
a001158      0.674453  0.735767  0.814345  0.789474  0.735767  0.719874   
a005594      0.613139  0.674453  0.698010  0.736842  0.551825  0.764866   
a007033      0.785714  0.714286  0.632456  0.613139  0.571429  0.524142   
...               ...       ...       ...       ...       ...       ...   
d000922      0.722806  0.677631  0.885714  0.775567  0.677631  0.762443   
d010626      0.681385  0.628971  0.729293  0.809858  0.628971  0.769231   
d001143      0.674453  0.735767  0.814345  0.789474  0.735767  0.719874   
b004853      0.674453  0.735767  0.814345  0.789474  0.735767  0.719874   
d000216      0.735767  0.797081  0.775567  0.842105  0.797081  0.674882   

TRAVELER_ID   a004008   a001563   a006197   a018176   a017831   a007230  \
TRAVELER_ID          

## 결과

### 'b007108' 관광객 여행지 추천

#### 'b007108' 정보
- 30대
- 남성
- INTJ
- 도시 중간선호
- 당일여행 중간선호
- 새로운지역 중간선호
- 편하지만 비싼숙소 조금선호
- 휴식휴양 조금선호
- 알려진 여행지 중간선호
- 계획 높은 선호

In [ ]:
like_city.loc[like_city['TRAVELER_ID']=='b007108']

,TRAVELER_ID,선호여행1,선호여행2,선호여행3
989,b007108,제주특별자치도,부산광역시,속초시


#### 유사한 5명의 관광객

In [ ]:
ex_target_similarity = similarity_df.loc['b007108']

# 자기 자신을 제외하고, 가장 유사도가 높은 상위 5명의 사용자 찾기
ex_top_5_similar_ids = ex_target_similarity.drop('b007108').nlargest(5).index
ex_top_5_similar_ids.values

array(['a006306', 'a018013', 'b016467', 'a005193', 'a006581'],
      dtype=object)

#### 'b007108'관광객과 유사 5명 관광객 성향 비교

In [ ]:
traveler_rev.loc[traveler_rev.index.isin(['b007108', 'a006306', 'a018013', 'b016467', 'a005193', 'a006581'])]

,TRAVELER_ID,연령,성별_남,성별_여,자연,도시,숙박,당일,익숙한,새로운,편하지만비싼숙소,불편하지만저렴한숙소,휴양휴식,체험활동,잘알려지지않은,알려진,계획에따른,상황에따른,사진촬영중요하지않음,사진촬영중요,MBTI_ENFJ,MBTI_ENFP,MBTI_ENTJ,MBTI_ENTP,MBTI_ESFJ,MBTI_ESFP,MBTI_ESTJ,MBTI_ESTP,MBTI_INFJ,MBTI_INFP,MBTI_INTJ,MBTI_INTP,MBTI_ISFJ,MBTI_ISFP,MBTI_ISTJ,MBTI_ISTP


### 추천결과

In [ ]:
# 사용자 ID 설정 (예시)
target_user_id = 'b007108'

# 여행지 추천 실행
recommended_cities_for_user = recommend_for_user(target_user_id, traveler_rev, like_city)
print(recommended_cities_for_user)

['인천광역시', '가평군', '원주시', '양평군', '상주시', '사천시', '서울특별시', '강릉시', '순창군']
